In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import gc

In [2]:
Data = pd.read_csv("./Source/CarOR_VanAvailability_MSOA_AnalysisCombined.csv",index_col=['PowerLiftRatio,Perc','Middle layer Super Output Areas Code'])
# CSV file taken from 'https://www.ons.gov.uk/datasets/TS045/editions/2021/versions/4/filter-outputs/21f5ef3b-68bf-44ee-b610-f654febc6e8d#get-data'

In [3]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 261504 entries, (100, 'E02000001') to (170, 'W02000428')
Data columns (total 11 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   1 car or van in household            261504 non-null  int64  
 1   2 cars or vans in household          261504 non-null  int64  
 2   3 or more cars or vans in household  261504 non-null  int64  
 3   No cars or vans in household         261504 non-null  int64  
 4   TotalHousehold                       261504 non-null  int64  
 5   TotalCars                            261504 non-null  int64  
 6   PowerProfile_kWe                     261504 non-null  object 
 7   NumberOfCars                         261504 non-null  object 
 8   TotalCarsCharged                     261504 non-null  float64
 9   EV_SufficiencyFlag                   261504 non-null  object 
 10  EV_ChargingPenetrationRate           261504 non-nul

In [4]:
DropColumns = ['1 car or van in household', '2 cars or vans in household','3 or more cars or vans in household', 'No cars or vans in household',
                'TotalHousehold', 'TotalCars', 'PowerProfile_kWe', 'NumberOfCars','TotalCarsCharged', 'EV_SufficiencyFlag']
Data.drop(columns=DropColumns,inplace=True)

In [5]:
gc.collect()

0

In [6]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 261504 entries, (100, 'E02000001') to (170, 'W02000428')
Data columns (total 1 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   EV_ChargingPenetrationRate  261504 non-null  float64
dtypes: float64(1)
memory usage: 3.5 MB


In [7]:
SHAPEFILE = './ShapeFiles/MSOA_2021_EW_BFE_V7_-121704356585247693/MSOA_2021_EW_BFE_V7.shp'
GeoDF = gpd.read_file(SHAPEFILE)[['MSOA21CD', 'geometry']]
GeoDF.columns = ['MSOA_21_Code', 'geometry']
GeoDF.head()

,MSOA_21_Code,geometry
0,E02000001,"POLYGON ((532153.703 182165.155, 532158.250 18..."
1,E02000002,"POLYGON ((548881.304 190819.980, 548881.125 19..."
2,E02000003,"POLYGON ((548958.555 189072.176, 548954.517 18..."
3,E02000004,"POLYGON ((551550.056 187364.705, 551528.633 18..."
4,E02000005,"POLYGON ((549237.051 187627.941, 549241.319 18..."


In [8]:
gc.collect()

0

In [9]:
GeoDF.info(memory_usage='deep')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7264 entries, 0 to 7263
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   MSOA_21_Code  7264 non-null   object  
 1   geometry      7264 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 525.1 KB


In [10]:
# EVPenetrationRateBins = np.arange(0,101,10)
# s = Data.loc[120]["EV_ChargingPenetrationRate"].value_counts(bins=EVPenetrationRateBins,sort=False)

In [11]:
# sdf = pd.DataFrame(s).transpose()
# sdf

In [12]:
def PlotFigure(DF,UpLift):
    Title = f'EV charging penetration MSOA\nPower Uplift of +{UpLift-100}%.'
    col = 'EV_ChargingPenetrationRate'
    vmin = DF[col].min()
    vmax = DF[col].max()
    cmap = 'RdYlGn'
    fig, ax = plt.subplots(1,figsize=(9,16),dpi=1500)
    ax.axis('off')
    DF.plot(column=col, ax=ax, edgecolor='white', linewidth=0.01, cmap=cmap)
    ax.set_title(Title, fontdict={'fontsize': '14', 'fontweight': '3'})# Add a title
    sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=0, vmax=100), cmap=cmap)# Create colorbar as a legend
    # Empty array for the data range
    sm._A = []
    cbaxes = fig.add_axes([0.18,0.35,0.02,0.2])
    cbar = fig.colorbar(sm, cax=cbaxes)
    plt.savefig(f"./Pictures/EV_Charging_Penetration_MSOA_{UpLift}.png")
    plt.close('all')
    gc.collect()

In [13]:
PowerLifts = set()
for item in Data.index:
    PowerLifts.add(item[0])
PowerLifts = sorted(PowerLifts)

In [17]:
gc.collect()
for ChargeUpLift in tqdm(PowerLifts[4::5]):
    SelectedData = Data.loc[ChargeUpLift].reset_index()
    MergedDF = pd.merge(left=GeoDF, right=SelectedData, how='left', left_on='MSOA_21_Code', right_on='Middle layer Super Output Areas Code')
    DropColumns = ['Middle layer Super Output Areas Code']
    MergedDF.drop(columns=DropColumns,inplace=True)
    PlotFigure(MergedDF,ChargeUpLift)
    gc.collect()

100%|████████████████████████████████████████████████████████████████████| 7/7 [02:35<00:00, 22.17s/it]
